In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive
!ls

/content/drive/My Drive
'10_Sheely Tira Maharani_Reading Assignment 1.pdf'
'13_Logistic Regression.gsheet'
'2023-12-27 11-41-31.mkv'
'210_220_KELOMPOK 17_2023A_KLASIFIKASI JENIS KELAMIN ULAR.gsheet'
'23031554210_Sheely Tira Maharani_23A.png'
'23031554210_Sheely Tira Maharani_23A_Tugas 5 dan 6.pdf'
'23031554210_Sheely Tira Maharani_Tugas 10.pdf'
'23031554210_Sheely Tira Maharani_Tugas Video.mp4'
'23031554210_Sheely Tira Maharani_UTS Bahasa Indonesia.pdf'
'23031554210_Sheely Tira Maharani_UTS Pancasila.pdf'
'23201554210_Sheely Tira M_2023A.pdf'
'23201554210_Sheely Tira M_23A.pdf'
'23201554210_Sheely Tira Maharani_2023A.pdf'
'23201554210_Sheely Tira Maharani_23A.pdf'
 7.png
 8.png
 all_imdb_reviews.csv
 all_imdb_reviews.gsheet
 all_imdb_reviews.xlsx
 aspirasi.gsheet
'basis data kelompok 5'
'CamScanner 03-19-2024 11.21.pdf'
 Classroom
'code_boston_pca&fa.R'
'Colab Notebooks'
'Conference-template-A4 (7).gdoc'
 Conference-template-A4.gdoc
'DAA tugas 2.gdoc'
'data project kelompok 10'
 datase

# Import Libraties/ Functions

In [8]:
import os
import numpy as np
import cv2
import joblib
from skimage.feature import local_binary_pattern, hog
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

# Import Dataset

In [16]:
train_dir = "/content/drive/MyDrive/dataset_pcd/train"
test_dir = "/content/drive/MyDrive/dataset_pcd/test"

In [17]:
def load_dataset(root_dir):
    X, y = [], []
    for label in sorted(os.listdir(root_dir)):
        class_dir = os.path.join(root_dir, label)
        if not os.path.isdir(class_dir):
            continue
        for f in os.listdir(class_dir):
            path = os.path.join(class_dir, f)
            try:
                feat = extract_features(path)
                X.append(feat)
                y.append(label)
            except Exception as e:
                print(f"Skip {path}: {e}")
    print(f"Loaded {len(X)} samples from {root_dir}")
    return np.array(X, dtype="float32"), np.array(y)

# Ekstraksi Fitur

In [18]:
radius = 1
n_points = 8 * radius
METHOD = 'uniform'


In [19]:
def extract_lbp(image):
    lbp = local_binary_pattern(image, n_points, radius, METHOD)
    (hist, _) = np.histogram(lbp.ravel(),
                             bins=np.arange(0, n_points + 3),
                             range=(0, n_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist

In [20]:
def extract_hog(image):
    features = hog(image, orientations=9, pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2), block_norm='L2-Hys', transform_sqrt=True)
    return features

In [21]:
def extract_features(img_path):
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Image {img_path} could not be read.")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (48, 48))
    lbp_feat = extract_lbp(gray)
    hog_feat = extract_hog(gray)
    return np.hstack([lbp_feat, hog_feat])

# Load Dataset

In [22]:
X_train, y_train = load_dataset(train_dir)
X_test, y_test = load_dataset(test_dir)

Loaded 28749 samples from /content/drive/MyDrive/dataset_pcd/train
Loaded 7188 samples from /content/drive/MyDrive/dataset_pcd/test


# Standarisasi Fitur

In [23]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Model SVM

In [ ]:
svm_model = SVC(kernel='rbf', C=10, gamma=0.001)
svm_model.fit(X_train, y_train)

In [ ]:
y_pred = svm_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


In [ ]:
joblib.dump(svm_model, "svm_model.joblib")
joblib.dump(scaler, "scaler.joblib")
print("Model dan scaler berhasil disimpan.")
